In [2]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from unicodedata import normalize

from src.data import cleandata

In [3]:
tqdm.pandas()

path_interim = "../data/interim/pjud"
path_processed = "../data/processed/pjud"

/Users/jalvaradoruiz/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# DATA de TRIBUNALES OBTENIDAS DEL COT

In [4]:
# Analizo contra los datos extraidos en COT
df_tribunales = pd.read_feather(f"{path_processed}/ListadoTribunales_feather")
tribunales = df_tribunales['TRIBUNAL'].unique()

# DATA INVENTARIO

In [5]:
df_inventario = pd.read_feather(f"{path_interim}/Inventario_feather")
df_inventario.head()

,index,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,COMPETENCIA,TIPO CAUSA,COD. MATERIA,MATERIA,TIPO ULT. DILIGENCIA,FECHA ULT. DILIGENCIA,FECHA INGRESO
0,0,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,123-2016,ORAL,Ordinaria,856,APROPIACION INDEBIDA ART.470 N°1,AUDIENCIA,2019-10-31,2016-04-29
1,1,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,159-2019,ORAL,Ordinaria,7037,TRAFICO DE PEQUEÑA CANTIDADE (ART. 4).,RESOLUCION,2019-12-31,2019-05-28
2,2,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,160-2019,ORAL,Ordinaria,10011,POSESION TENENCIA O PORTE DE MUN Y SUST QUIMICAS,RESOLUCION,2019-11-21,2019-05-28
3,3,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,176-2019,ORAL,Ordinaria,12051,CONTRAB. INFRAC A LA ORD. DE ADUAN ART 168. LE...,RESOLUCION,2019-12-31,2019-06-06
4,4,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL DE ARICA,196-2013,ORAL,Ordinaria,7007,TRAFICO ILICITO DE DROGA (ART. 3).,RESOLUCION,2019-09-10,2013-08-05


In [6]:
# Normalizo nombres

df_inventario['TRIBUNAL'] = df_inventario['TRIBUNAL'].progress_apply(cleandata.cambio_nombre_juzgados)

100%|██████████| 1786310/1786310 [00:10<00:00, 176504.07it/s]


In [7]:
# Verifico si coinciden los nombres e imprimo los que estan en conflicto

for tribunal in df_inventario['TRIBUNAL'].unique():
    if not tribunal in tribunales:
        print(tribunal)

## Data relacionada a delitos

In [8]:
df_delitos = pd.read_feather(f"{path_processed}/Delitos_feather")
df_delitos.head()

,index,COD. MATERIA,MATERIA,TIPOLOGIA MATERIA,VIGENCIA MATERIA
0,0,101,QUEBRANTAMIENTO. ART. 90,LIBRO I TITULO IV QUEBRANTAMIENTO DE SENTEN...,VIGENTE
1,1,202,SECUESTRO. ART. 141,LIBRO II TITULO III CRIMENES Y SIMPLES DELIT...,VIGENTE
2,2,203,SUSTRACCION DE MENORES. ART. 142,LIBRO II TITULO III CRIMENES Y SIMPLES DELIT...,VIGENTE
3,3,204,VIOLACION DE MORADA. ART.144,LIBRO II TITULO III CRIMENES Y SIMPLES DELIT...,VIGENTE
4,4,205,"APERTURA , REGISTRO O INTERCEPTACION DE CORRE...",LIBRO II TITULO III CRIMENES Y SIMPLES DELIT...,VIGENTE


In [9]:
codigos_cot = df_delitos['COD. MATERIA'].unique()
print(f"Existen {len(codigos_cot)} codigos de delitos")

Existen 541 codigos de delitos


## Delitos en INVENTARIO

In [10]:
codigos_delitos = df_inventario['COD. MATERIA'].unique()

In [11]:
len(codigos_delitos)

458

In [12]:
codigos_no_encontrados = []
for delito in codigos_delitos:
    if not delito in codigos_cot:
        codigos_no_encontrados.append(delito)

In [13]:
codigos_no_encontrados

[13001, -31075, 4014, 4013, 4010, 4011, 0]

In [16]:
codigos_sin_registro = df_inventario[df_inventario['COD. MATERIA'].isin(codigos_no_encontrados)]

# Analizar caso a caso:

#### Caso Codigo 13001 -> LESIONES LEVES
Este codigo debe tratarse como 13036 identificado como: LESIONES LEVES 494 Nº 5 CÓDIGO PENAL

In [17]:
df_inventario['COD. MATERIA'] = df_inventario['COD. MATERIA'].replace(13001,13036)

### Codigos 4011, 4014, 4013, 4010, -31075, 0
codigos No vigentes, mal cargados en sistemas 

In [18]:
filtro_codigo_4011 = df_inventario[df_inventario['COD. MATERIA']==4011]
filtro_codigo_4014 = df_inventario[df_inventario['COD. MATERIA']==4014]
filtro_codigo_4013 = df_inventario[df_inventario['COD. MATERIA']==4013]
filtro_codigo_4010 = df_inventario[df_inventario['COD. MATERIA']==4010]
filtro_codigo_31075 = df_inventario[df_inventario['COD. MATERIA']==31075]
filtro_codigo_0 = df_inventario[df_inventario['COD. MATERIA']==0]

In [19]:
df_inventario.drop(filtro_codigo_4011.index, axis = 0, inplace = True)
df_inventario.drop(filtro_codigo_4014.index, axis = 0, inplace = True)
df_inventario.drop(filtro_codigo_4013.index, axis = 0, inplace = True)
df_inventario.drop(filtro_codigo_4010.index, axis = 0, inplace = True)
df_inventario.drop(filtro_codigo_31075.index, axis = 0, inplace = True)
df_inventario.drop(filtro_codigo_0.index, axis = 0, inplace = True)

In [21]:
# Directorio donde se guardaran archivos feather
df_inventario.reset_index(inplace = True)
os.makedirs(path_processed, exist_ok = True)  
# Guardamos dataset como archivo feather
df_inventario.to_feather(f'{path_processed}/Inventario_feather')